In [ ]:
!pip install pip install html-table-parser-python3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from urllib.request import urlopen
from urllib.error import URLError
from urllib.parse import urlparse
import csv
import sys
import time
import os
import unicodedata
from bs4 import BeautifulSoup, SoupStrainer
import requests
from html_table_parser.parser import HTMLTableParser
import pandas as pd
import numpy as np
from itertools import chain

In [ ]:
home_url = 'http://www.howstat.com/cricket/Statistics/Matches/MatchListMenu.asp'

In [ ]:
season_range = list(range(2008, 2023))
print(season_range)

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


In [ ]:
season_urls = []

for i in season_range:
    url = f'http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season={i}'
    season_urls.append(url)

print(*season_urls, sep='\n')

http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2008
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2009
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2010
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2011
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2012
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2013
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2014
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2015
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2016
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2017
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2018
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2019
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Season=2020
http://www.howstat.com/cricket/Statistics/IPL/MatchList.asp?Seas

In [ ]:
base = 'http://www.howstat.com/cricket/Statistics/IPL/'
all_seasons_data = []
for u in season_range[4:]:
    url = base + f'MatchList.asp?Season={u}'
    print(url)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    find_table = soup.find('table', class_='TableLined')
    rows = find_table.find_all('tr')
    rows = rows[1:-1]
    all_matches_url = []
    for r in range(len(rows)):
        all_tds = rows[r].find_all('td')
        anchor_tag = all_tds[1].find('a', href=True)
        match_code = anchor_tag['href']
        match_url = base + match_code
        all_matches_url.append(match_url)
    
    all_matches = []
    for m in all_matches_url:
        master_list = []
        res = requests.get(m)
        soup = BeautifulSoup(res.text, "html.parser")
        match_details = str(soup.find('td', class_='ScoreCardBannerIPL').text).strip()
        match_between = match_details.partition('-')[2].strip('\r\t\n-').partition(' -')[0]
        home_team = match_between.partition(' v ')[0]
        master_list.append(home_team)
        away_team = match_between.partition(' v ')[2]
        master_list.append(away_team)
        table = soup.find('table', class_='Table_D')
        rows = table.find_all('tr')
        for row in rows[:-1]:
            print(row)
            try:
                col_value = str(row.find('td', class_='TextBlack8').text).strip()
                master_list.append(col_value)
            except AttributeError:
                continue
        all_matches.append(master_list)
        time.sleep(5)
    time.sleep(20)
    all_seasons_data.append(all_matches)

In [ ]:
all_seasons_data_until_2011 = all_seasons_data

In [ ]:
all_seasons_data_until_2011.append(all_seasons_data)

In [ ]:
len(all_seasons_data_until_2011)

5

In [ ]:
flat_list = list(chain(*all_seasons_data))

In [ ]:
len(flat_list)

706

In [ ]:
flat_list_1 = list(chain(*all_seasons_data_until_2011[:-1]))

In [ ]:
len(flat_list_1)

252

In [ ]:
full_list = flat_list + flat_list_1

In [ ]:
len(full_list)

958

In [ ]:
len(full_list[0])

9

In [ ]:
headers = ['home', 'away', 'date', 'season','venue', 'type', 'toss', 'winner', 'man_of_match', 'points']

In [ ]:
df = pd.DataFrame(full_list, columns=headers)

In [ ]:
df

,home,away,date,season,venue,type,toss,winner,man_of_match,points
0,Chennai Super Kings,Mumbai Indians,"4th April, 2012",2012 - 1st Match,"MA Chidambaram Stadium, Chennai","20 Overs, Day/Night Match",Mumbai Indians won the toss and elected to field,Mumbai Indians won by 8 wickets,Richard Levi,None
1,Kolkata Knight Riders,Delhi Daredevils,"5th April, 2012",2012 - 2nd Match,"Eden Gardens, Kolkata","20 Overs, Day/Night Match",Delhi Daredevils won the toss and elected to f...,Delhi Daredevils won by 8 wickets,Irfan Pathan,"Delhi Daredevils 2, Kolkata Knight Riders 0"
2,Mumbai Indians,Pune Warriors,"6th April, 2012",2012 - 3rd Match,"Wankhede Stadium, Mumbai","20 Overs, Day/Night Match",Mumbai Indians won the toss and elected to field,Pune Warriors won by 28 runs,Steve Smith,None
3,Rajasthan Royals,Kings XI Punjab,"6th April, 2012",2012 - 4th Match,"Sawai Mansingh Stadium, Jaipur","20 Overs, Day/Night Match",Kings XI Punjab won the toss and elected to field,Rajasthan Royals won by 31 runs,Ajinkya Rahane,None
4,Royal Challengers Bangalore,Delhi Daredevils,"7th April, 2012",2012 - 5th Match,"M Chinnaswamy Stadium, Bengaluru, Bangalore","20 Overs, Day/Night Match",Delhi Daredevils won the toss and elected to f...,Royal Challengers Bangalore won by 20 runs,AB de Villiers,None
...,...,...,...,...,...,...,...,...,...,...
953,Kolkata Knight Riders,Mumbai Indians,"22nd May, 2011",2011 - 70th Match,"Eden Gardens, Kolkata","20 Overs, Day/Night Match",Mumbai Indians won the toss and elected to field,Mumbai Indians won by 5 wickets,James Franklin,None
954,Royal Challengers Bangalore,Chennai Super Kings,"24th May, 2011",2011 - 1st Qualifying Final,"Wankhede Stadium, Mumbai","20 Overs, Day/Night Match",Chennai Super Kings won the toss and elected t...,Chennai Super Kings won by 6 wickets,Suresh Raina,None
955,Kolkata Knight Riders,Mumbai Indians,"25th May, 2011",2011 - Elimination Final,"Wankhede Stadium, Mumbai","20 Overs, Day/Night Match",Mumbai Indians won the toss and elected to field,Mumbai Indians won by 4 wickets,Munaf Patel,None
956,Royal Challengers Bangalore,Mumbai Indians,"27th May, 2011",2011 - 2nd Qualifying Final,"MA Chidambaram Stadium, Chennai","20 Overs, Day/Night Match",Mumbai Indians won the toss and elected to field,Royal Challengers Bangalore won by 43 runs,Chris Gayle,None


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
cd /content/drive/MyDrive/Colab/MIPT/DevOps/Bharani

/content/drive/MyDrive/Colab/MIPT/DevOps/Bharani


In [ ]:
df = pd.read_csv('ipl_data.csv', index_col=[0])

In [ ]:
df

,home,away,date,season,venue,type,toss,winner,man_of_match,points
0,Chennai Super Kings,Mumbai Indians,"4th April, 2012",2012 - 1st Match,"MA Chidambaram Stadium, Chennai","20 Overs, Day/Night Match",Mumbai Indians won the toss and elected to field,Mumbai Indians won by 8 wickets,Richard Levi,NaN
1,Kolkata Knight Riders,Delhi Daredevils,"5th April, 2012",2012 - 2nd Match,"Eden Gardens, Kolkata","20 Overs, Day/Night Match",Delhi Daredevils won the toss and elected to f...,Delhi Daredevils won by 8 wickets,Irfan Pathan,"Delhi Daredevils 2, Kolkata Knight Riders 0"
2,Mumbai Indians,Pune Warriors,"6th April, 2012",2012 - 3rd Match,"Wankhede Stadium, Mumbai","20 Overs, Day/Night Match",Mumbai Indians won the toss and elected to field,Pune Warriors won by 28 runs,Steve Smith,NaN
3,Rajasthan Royals,Kings XI Punjab,"6th April, 2012",2012 - 4th Match,"Sawai Mansingh Stadium, Jaipur","20 Overs, Day/Night Match",Kings XI Punjab won the toss and elected to field,Rajasthan Royals won by 31 runs,Ajinkya Rahane,NaN
4,Royal Challengers Bangalore,Delhi Daredevils,"7th April, 2012",2012 - 5th Match,"M Chinnaswamy Stadium, Bengaluru, Bangalore","20 Overs, Day/Night Match",Delhi Daredevils won the toss and elected to f...,Royal Challengers Bangalore won by 20 runs,AB de Villiers,NaN
...,...,...,...,...,...,...,...,...,...,...
953,Kolkata Knight Riders,Mumbai Indians,"22nd May, 2011",2011 - 70th Match,"Eden Gardens, Kolkata","20 Overs, Day/Night Match",Mumbai Indians won the toss and elected to field,Mumbai Indians won by 5 wickets,James Franklin,NaN
954,Royal Challengers Bangalore,Chennai Super Kings,"24th May, 2011",2011 - 1st Qualifying Final,"Wankhede Stadium, Mumbai","20 Overs, Day/Night Match",Chennai Super Kings won the toss and elected t...,Chennai Super Kings won by 6 wickets,Suresh Raina,NaN
955,Kolkata Knight Riders,Mumbai Indians,"25th May, 2011",2011 - Elimination Final,"Wankhede Stadium, Mumbai","20 Overs, Day/Night Match",Mumbai Indians won the toss and elected to field,Mumbai Indians won by 4 wickets,Munaf Patel,NaN
956,Royal Challengers Bangalore,Mumbai Indians,"27th May, 2011",2011 - 2nd Qualifying Final,"MA Chidambaram Stadium, Chennai","20 Overs, Day/Night Match",Mumbai Indians won the toss and elected to field,Royal Challengers Bangalore won by 43 runs,Chris Gayle,NaN


In [ ]:
winner = lambda x: x.partition(' won')[0]
toss = lambda x: x.partition(' won')[0]
type_of_match = lambda x:x.partition(', ')[2]
change_season = lambda x: 'league' if 'Match' in x else 'knockout'

In [ ]:
df = pd.read_csv('ipl_data.csv', index_col=[0])
df.drop(['points'], axis=1, inplace=True)
df['winner'] = df['winner'].apply(winner)
df['toss'] = df['toss'].apply(toss)
df['type'] = df['type'].apply(type_of_match)
df['season'] = df['season'].apply(change_season)
df['winner'].replace("Match abandoned", "No result", inplace=True)
df['outcome'] = np.where(df['home'] == df['winner'], 1, 0)
df.rename(columns={"season": "match_type", "type": "time_of_match"}, inplace=True)
df

,home,away,date,match_type,venue,time_of_match,toss,winner,man_of_match,outcome
0,Chennai Super Kings,Mumbai Indians,"4th April, 2012",league,"MA Chidambaram Stadium, Chennai",Day/Night Match,Mumbai Indians,Mumbai Indians,Richard Levi,0
1,Kolkata Knight Riders,Delhi Daredevils,"5th April, 2012",league,"Eden Gardens, Kolkata",Day/Night Match,Delhi Daredevils,Delhi Daredevils,Irfan Pathan,0
2,Mumbai Indians,Pune Warriors,"6th April, 2012",league,"Wankhede Stadium, Mumbai",Day/Night Match,Mumbai Indians,Pune Warriors,Steve Smith,0
3,Rajasthan Royals,Kings XI Punjab,"6th April, 2012",league,"Sawai Mansingh Stadium, Jaipur",Day/Night Match,Kings XI Punjab,Rajasthan Royals,Ajinkya Rahane,1
4,Royal Challengers Bangalore,Delhi Daredevils,"7th April, 2012",league,"M Chinnaswamy Stadium, Bengaluru, Bangalore",Day/Night Match,Delhi Daredevils,Royal Challengers Bangalore,AB de Villiers,1
...,...,...,...,...,...,...,...,...,...,...
953,Kolkata Knight Riders,Mumbai Indians,"22nd May, 2011",league,"Eden Gardens, Kolkata",Day/Night Match,Mumbai Indians,Mumbai Indians,James Franklin,0
954,Royal Challengers Bangalore,Chennai Super Kings,"24th May, 2011",knockout,"Wankhede Stadium, Mumbai",Day/Night Match,Chennai Super Kings,Chennai Super Kings,Suresh Raina,0
955,Kolkata Knight Riders,Mumbai Indians,"25th May, 2011",knockout,"Wankhede Stadium, Mumbai",Day/Night Match,Mumbai Indians,Mumbai Indians,Munaf Patel,0
956,Royal Challengers Bangalore,Mumbai Indians,"27th May, 2011",knockout,"MA Chidambaram Stadium, Chennai",Day/Night Match,Mumbai Indians,Royal Challengers Bangalore,Chris Gayle,1


In [ ]:
df.man_of_match.value_counts()

AB de Villiers      25
Chris Gayle         22
David Warner        18
Rohit Sharma        18
M S Dhoni           17
                    ..
Shimron Hetmeyer     1
Khaleel Ahmed        1
Keemo Paul           1
Imran Tahir          1
James Franklin       1
Name: man_of_match, Length: 263, dtype: int64

In [ ]:
### code for Scraping ###

'''
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

def main():
    years_we_want = [*range(2023, 2022, -1)]
    full_matches = []
    home_url = "https://fbref.com/en/comps/11/Serie-A-Stats"
    # home_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

    for year in years_we_want:
        year_data = requests.get(home_url)
        year_soup = BeautifulSoup(year_data.text, "lxml")

        points_table = year_soup.select("table.stats_table")[0]
        all_club_links = points_table.find_all("a")

        href_of_clubs = []
        for club_link in all_club_links:
            href_of_clubs.append(club_link.get("href"))

        club_links = []
        for club_link in all_club_links:
            if "/squads" in club_link.get("href"):
                club_links.append(club_link.get("href"))

        full_club_links = []
        for club_link in club_links:
            full_club_links.append("https://fbref.com" + club_link)

        prev_link = year_soup.select("a.prev")[0]
        previous_season = prev_link.get("href")
        home_url = "https://fbref.com" + previous_season

        for club_url in full_club_links:
            club_data = requests.get(club_url)
            club_soup = BeautifulSoup(club_data.text, "lxml")
            club_name = club_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            all_matches_df = pd.read_html(club_data.text, match="Scores & Fixtures")[0]
            match_logs_links = club_soup.find_all("a")

            match_logs_links_href = []
            for match_logs_link in match_logs_links:
                match_logs_links_href.append(match_logs_link.get("href"))

            shooting_links = []
            for match_logs_link_href in match_logs_links_href:
                if (
                    match_logs_link_href
                    and "all_comps/shooting" in match_logs_link_href
                ):
                    shooting_links.append(match_logs_link_href)

            full_shooting_links = []
            for shooting_link in shooting_links:
                full_shooting_links.append("https://fbref.com" + shooting_link)

            data_for_shooting_links = requests.get(full_shooting_links[0])
            try:
                shooting_df = pd.read_html(
                    data_for_shooting_links.text, match="Shooting"
                )[0]
                shooting_df.columns = shooting_df.columns.droplevel()
                team_match_shoot_df = pd.merge(
                    all_matches_df,
                    shooting_df[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]],
                    on="Date",
                    how="left",
                )

            except ValueError:

                continue

            team_match_shoot_df = team_match_shoot_df[
                team_match_shoot_df["Comp"] == "Serie A"
            ]
            team_match_shoot_df["Season"] = year
            team_match_shoot_df["Team"] = club_name
            full_matches.append(team_match_shoot_df)
            time.sleep(2)

    matches_df = pd.concat(full_matches)
    matches_df.columns = [col.lower() for col in matches_df.columns]
    matches_df.to_csv("matches_serie-a.csv")


if __name__ == "__main__":
    main()